<a href="https://colab.research.google.com/github/vincimech010233/MachineLearningBasics/blob/master/material_fatigue_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Paso 1: Importar las bibliotecas necesarias
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Paso 2: Generar y guardar los datos simulados en un archivo CSV
np.random.seed(0)
n_samples = 1000
density = np.random.uniform(2.5, 7.5, n_samples)
tensile_strength = np.random.uniform(300, 1000, n_samples)
operating_temp = np.random.uniform(-50, 150, n_samples)
cycles_to_failure = np.random.normal(50000, 10000, n_samples)
cycles_to_failure = np.where(cycles_to_failure < 0, 0, cycles_to_failure)

data = pd.DataFrame({
    'Density': density,
    'Tensile_Strength': tensile_strength,
    'Operating_Temp': operating_temp,
    'Cycles_To_Failure': cycles_to_failure
})

csv_file = 'material_fatigue_data.csv'
data.to_csv(csv_file, index=False)

# Paso 3: Cargar y preprocesar los datos
data = pd.read_csv(csv_file)
X = data.drop('Cycles_To_Failure', axis=1)
y = data['Cycles_To_Failure']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

scaler_X = StandardScaler()
scaler_y = MinMaxScaler()  # Escalador para la variable objetivo

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).ravel()
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1)).ravel()

# Paso 4: Crear y entrenar el modelo de regresión
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train_scaled, y_train_scaled, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Paso 5: Evaluar el modelo
loss = model.evaluate(X_test_scaled, y_test_scaled)
print(f'Pérdida en el conjunto de prueba: {loss}')

# Predicciones (opcional)
predictions = model.predict(X_test_scaled)


Epoch 1/100
20/20 [==============================] - 4s 51ms/step - loss: 0.0677 - val_loss: 0.0404
Epoch 2/100
20/20 [==============================] - 0s 9ms/step - loss: 0.0328 - val_loss: 0.0303
Epoch 3/100
20/20 [==============================] - 0s 9ms/step - loss: 0.0296 - val_loss: 0.0283
Epoch 4/100
20/20 [==============================] - 0s 7ms/step - loss: 0.0269 - val_loss: 0.0264
Epoch 5/100
20/20 [==============================] - 0s 7ms/step - loss: 0.0253 - val_loss: 0.0249
Epoch 6/100
20/20 [==============================] - 0s 6ms/step - loss: 0.0248 - val_loss: 0.0246
Epoch 7/100
20/20 [==============================] - 0s 8ms/step - loss: 0.0228 - val_loss: 0.0237
Epoch 8/100
20/20 [==============================] - 0s 7ms/step - loss: 0.0222 - val_loss: 0.0226
Epoch 9/100
20/20 [==============================] - 0s 13ms/step - loss: 0.0219 - val_loss: 0.0245
Epoch 10/100
20/20 [==============================] - 0s 7ms/step - loss: 0.0219 - val_loss: 0.0254
Epoch 1

In [7]:
# Crear nuevos datos
nuevos_datos = pd.DataFrame({
    'Density': [4.5],
    'Tensile_Strength': [600],
    'Operating_Temp': [50]
})

# Escalar los nuevos datos
nuevos_datos_escalados = scaler_X.transform(nuevos_datos)

# Hacer la predicción
prediccion_escalada = model.predict(nuevos_datos_escalados)

# Reescalar la predicción a la escala original
prediccion_original = scaler_y.inverse_transform(prediccion_escalada)

print(f"Predicción del número de ciclos hasta la falla: {prediccion_original[0][0]}")


1/1 [==============================] - 0s 46ms/step
Predicción del número de ciclos hasta la falla: 53309.31640625
